# Imminent ICU Admission Classifier with Logistic Regression

## Imports & Inits

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../../')

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

import pickle
import scipy

import pandas as pd
import numpy as np
np.set_printoptions(precision=4)

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
%matplotlib inline

from pathlib import Path

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from scikitplot.metrics import *

from utils.data_utils import set_group_splits
from utils.metrics import BinaryAvgMetrics, get_best_model
from utils.plots import *

In [2]:
from args import args
vars(args)

{'path': PosixPath('../data'),
 'workdir': PosixPath('../data/workdir/lr'),
 'figdir': PosixPath('../data/workdir/lr/figures'),
 'structured_csv': PosixPath('../data/modelready_structured.csv'),
 'unstructured_csv': PosixPath('../data/modelready_unstructured.csv'),
 'mm_csv': PosixPath('../data/modelready_mm.csv'),
 'modeldir': PosixPath('../data/workdir/lr/models')}

In [3]:
seeds = [643] + list(range(127,227))

In [7]:
notes_common = pd.read_csv(args.unstructured_csv)
notes_common = notes_common[notes_common['imi_adm_label'] != -1].reset_index(drop=True)

vitals_common = pd.read_csv(args.structured_csv)
vitals_common = vitals_common[vitals_common['imi_adm_label'] != -1].reset_index(drop=True)

mm_notes_vitals = pd.read_csv(args.mm_csv)
mm_notes_vitals = mm_notes_vitals[mm_notes_vitals['imi_adm_label'] != -1].reset_index(drop=True)

print(notes_common.shape, vitals_common.shape, mm_notes_vitals.shape)

(33870, 3) (205261, 10) (33870, 51)


In [8]:
seed = 643

In [9]:
df = set_group_splits(notes_common.copy(), group_col='hadm_id', seed=seed)

train_df = df[df['split'] == 'train']
test_df = df[df['split'] == 'test']

g = train_df.groupby(['imi_adm_label']).size().to_numpy()
print(f"Prevalence of positive class in training set:{(g[1]/g.sum())*100:0.1f}%")
g = test_df.groupby(['imi_adm_label']).size().to_numpy()
print(f"Prevalence of positive class in test set:{(g[1]/g.sum())*100:0.1f}%")

Prevalence of positive class in training set:23.0%
Prevalence of positive class in test set:19.9%


In [10]:
df = set_group_splits(vitals_common.copy(), group_col='hadm_id', seed=seed)

train_df = df[df['split'] == 'train']
test_df = df[df['split'] == 'test']

g = train_df.groupby(['imi_adm_label']).size().to_numpy()
print(f"Prevalence of positive class in training set:{(g[1]/g.sum())*100:0.1f}%")
g = test_df.groupby(['imi_adm_label']).size().to_numpy()
print(f"Prevalence of positive class in test set:{(g[1]/g.sum())*100:0.1f}%")

Prevalence of positive class in training set:18.2%
Prevalence of positive class in test set:20.0%


In [11]:
df = set_group_splits(mm_notes_vitals.copy(), group_col='hadm_id', seed=seed)

train_df = df[df['split'] == 'train']
test_df = df[df['split'] == 'test']

g = train_df.groupby(['imi_adm_label']).size().to_numpy()
print(f"Prevalence of positive class in training set:{(g[1]/g.sum())*100:0.1f}%")
g = test_df.groupby(['imi_adm_label']).size().to_numpy()
print(f"Prevalence of positive class in test set:{(g[1]/g.sum())*100:0.1f}%")

Prevalence of positive class in training set:23.0%
Prevalence of positive class in test set:19.9%


In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=60_000)
x_train = vectorizer.fit_transform(train_df['note'])
x_test = vectorizer.transform(test_df['note'])

with open(args.workdir/f'bi_gram_vital_643.pkl', 'wb') as f:
  pickle.dump(vectorizer, f)
  pickle.dump(x_note_train, f)
  pickle.dump(x_note_test, f)

In [ ]:
seed = 643
save = False

## Vitals Single Model Dev

In [ ]:
args.str_vital_cols = pickle.load(open(args.path/'str_vital_cols.pkl', 'rb'))
args.vital_cols=['hadm_id'] + args.str_vital_cols + ['note', 'imi_adm_label']

In [ ]:
full_common_vital = pd.read_csv(args.vital_csv, usecols=args.vital_cols)
full_common_vital = full_common_vital[full_common_vital['imi_adm_label'] != -1].reset_index(drop=True)
notes_common_vital = full_common_vital[['hadm_id', 'note', 'imi_adm_label']].copy().reset_index(drop=True)

print(notes_common_vital.shape, full_common_vital.shape)
notes_common_vital['hadm_id'].nunique(), full_common_vital['hadm_id'].nunique()

In [ ]:
# notes_common_vital.dropna(inplace=True)
# notes_common_vital.reset_index(drop=True, inplace=True)

# vital_hadms = set(full_common_vital['hadm_id'].unique()).intersection(notes_common_vital['hadm_id'].unique())

# full_common_vital = full_common_vital[full_common_vital['hadm_id'].isin(vital_hadms)].reset_index(drop=True)
# notes_common_vital = notes_common_vital[notes_common_vital['hadm_id'].isin(vital_hadms)].reset_index(drop=True)

# print(notes_common_vital.shape, full_common_vital.shape)
# notes_common_vital['hadm_id'].nunique(), full_common_vital['hadm_id'].nunique()

In [ ]:
# full_common_vital.dropna(inplace=True)
# full_common_vital.reset_index(drop=True, inplace=True)

In [ ]:
# full_common_vital[['note']] = full_common_vital.groupby('hadm_id')[['note']].ffill()

# full_common_vital.dropna(inplace=True)
# full_common_vital.reset_index(drop=True, inplace=True)

# x = pd.DataFrame(full_common_vital.isna().sum(), columns=['sum']).reset_index()
# x['sum'].sum()

### Notes only

In [ ]:
df = set_group_splits(notes_common_vital.copy(), group_col='hadm_id', seed=seed)

train_df = df[df['split'] == 'train']
test_df = df[df['split'] == 'test']

g = train_df.groupby(['imi_adm_label']).size().to_numpy()
print(f"Prevalence of positive class in training set:{(g[1]/g.sum())*100:0.1f}%")
g = test_df.groupby(['imi_adm_label']).size().to_numpy()
print(f"Prevalence of positive class in test set:{(g[1]/g.sum())*100:0.1f}%")

In [ ]:
# vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=60_000)
# x_note_train = vectorizer.fit_transform(train_df['note'])
# x_note_test = vectorizer.transform(test_df['note'])

# with open(args.workdir/f'bi_gram_vital_643.pkl', 'wb') as f:
#   pickle.dump(vectorizer, f)
#   pickle.dump(x_note_train, f)
#   pickle.dump(x_note_test, f)

In [ ]:
with open(args.workdir/f'vectordir/bi_gram_vital_643.pkl', 'rb') as f:
  vectorizer = pickle.load(f)
  x_note_train = pickle.load(f)
  x_note_test = pickle.load(f)
  
y_train, y_test = train_df['imi_adm_label'], test_df['imi_adm_label']
x_note_train.shape, x_note_test.shape, y_train.shape, y_test.shape

In [ ]:
clf = LogisticRegression(class_weight='balanced')
clf.fit(x_note_train, y_train)

prob = clf.predict_proba(x_note_test)
pos_prob = prob[:, 1]

labels = ['Delayed', 'Imminent']
label_test = [labels[i] for i in y_test]

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
plot_roc(label_test, prob, title='', ax=ax)
ax.set_xlabel('1 - Specificity')
ax.set_ylabel('Sensitivity')

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
plot_thresh_range(ax, y_test, pos_prob, lower=0.1, upper=0.81, n_vals=100)

if save:
  fig.savefig(args.figdir/f'{args.model}_notes_vital_metrics_vary.pdf', dpi=300)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
best_threshold = plot_thresh_metric(ax, y_test, pos_prob, lower=0.1, upper=0.81, n_vals=100)
ax.text(0.71, ax.get_ylim()[1] * 0.9, f'Optimum Threshold = {best_threshold[0]}', fontsize=12, color='b')
print(best_threshold)

if save:
  fig.savefig(args.figdir/f'{args.model}_notes_vital_threshold_guide.pdf', dpi=300)

In [ ]:
threshold = 0.41
pred = (pos_prob > threshold).astype(np.int64)
label_preds = [labels[i] for i in pred]
cm = confusion_matrix(y_test, pred)
tn,fp,fn,tp = cm[0][0],cm[0][1],cm[1][0],cm[1][1]
sensitivity = tp/(tp+fn)
specificity = tn/(tn+fp)
ppv = tp/(tp+fp)
npv = tn/(tn+fn)
f1 = (2*ppv*sensitivity)/(ppv+sensitivity)
auroc = roc_auc_score(y_test, pos_prob)

d = {
  'sensitivity': np.round(sensitivity, 3),
  'specificity': np.round(specificity, 3),
  'ppv': np.round(ppv, 3),
  'npv': np.round(npv, 3),
  'f1': np.round(f1, 3),
  'auroc': np.round(auroc, 3),
  'threshold': threshold,
}
metrics = pd.DataFrame(d.values(), index=d.keys(), columns=['Value'])
metrics

In [ ]:
fig, ax = plt.subplots(figsize=(11, 8))
plot_confusion_matrix(label_test, label_preds, x_tick_rotation=45, ax=ax, normalize=False)

In [ ]:
pos = 1220
control = 4898

### Notes and Vitals

In [ ]:
df = set_group_splits(full_common_vital.copy(), group_col='hadm_id', seed=seed)

train_df = df[df['split'] == 'train']
test_df = df[df['split'] == 'test']

g = train_df.groupby(['imi_adm_label']).size().to_numpy()
print(f"Prevalence of positive class in training set:{(g[1]/g.sum())*100:0.1f}%")
g = test_df.groupby(['imi_adm_label']).size().to_numpy()
print(f"Prevalence of positive class in test set:{(g[1]/g.sum())*100:0.1f}%")

In [ ]:
# vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=60_000)
# x_note_train = vectorizer.fit_transform(train_df['note'])
# x_note_test = vectorizer.transform(test_df['note'])

# with open(args.workdir/f'bi_gram_full_vital_643.pkl', 'wb') as f:
#   pickle.dump(vectorizer, f)
#   pickle.dump(x_note_train, f)
#   pickle.dump(x_note_test, f)

In [ ]:
with open(args.workdir/f'vectordir/bi_gram_vital_643.pkl', 'rb') as f:
  vectorizer = pickle.load(f)
  x_note_train = pickle.load(f)
  x_note_test = pickle.load(f)
  
y_train, y_test = train_df['imi_adm_label'], test_df['imi_adm_label']
x_note_train.shape, x_note_test.shape, y_train.shape, y_test.shape

In [ ]:
x_str_train, x_str_test = train_df[args.str_vital_cols].values, test_df[args.str_vital_cols].values
x_str_train.shape, x_str_test.shape

In [ ]:
x_train = scipy.sparse.hstack((x_str_train, x_note_train)).tocsr()
x_test = scipy.sparse.hstack((x_str_test, x_note_test)).tocsr()
x_train.shape, x_test.shape

In [ ]:
clf = LogisticRegression(class_weight='balanced')
clf.fit(x_train, y_train)

prob = clf.predict_proba(x_test)
pos_prob = prob[:, 1]

labels = ['Delayed', 'Imminent']
label_test = [labels[i] for i in y_test]

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
plot_roc(label_test, prob, title='', ax=ax)
ax.set_xlabel('1 - Specificity')
ax.set_ylabel('Sensitivity')

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
plot_thresh_range(ax, y_test, pos_prob, lower=0.1, upper=0.81, n_vals=100)

if save:
  fig.savefig(args.figdir/f'{args.model}_full_vital_metrics_vary.pdf', dpi=300)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
best_threshold = plot_thresh_metric(ax, y_test, pos_prob, lower=0.1, upper=0.81, n_vals=100)
ax.text(0.71, ax.get_ylim()[1] * 0.9, f'Optimum Threshold = {best_threshold[0]}', fontsize=12, color='b')
print(best_threshold)

if save:
  fig.savefig(args.figdir/f'{args.model}_full_vital_threshold_guide.pdf', dpi=300)

In [ ]:
threshold = 0.43
pred = (pos_prob > threshold).astype(np.int64)
label_preds = [labels[i] for i in pred]
cm = confusion_matrix(y_test, pred)
tn,fp,fn,tp = cm[0][0],cm[0][1],cm[1][0],cm[1][1]
sensitivity = tp/(tp+fn)
specificity = tn/(tn+fp)
ppv = tp/(tp+fp)
npv = tn/(tn+fn)
f1 = (2*ppv*sensitivity)/(ppv+sensitivity)
auroc = roc_auc_score(y_test, pos_prob)

d = {
  'sensitivity': np.round(sensitivity, 3),
  'specificity': np.round(specificity, 3),
  'ppv': np.round(ppv, 3),
  'npv': np.round(npv, 3),
  'f1': np.round(f1, 3),
  'auroc': np.round(auroc, 3),
  'threshold': threshold,
}
metrics = pd.DataFrame(d.values(), index=d.keys(), columns=['Value'])
metrics

In [ ]:
fig, ax = plt.subplots(figsize=(11, 8))
plot_confusion_matrix(label_test, label_preds, x_tick_rotation=45, ax=ax, normalize=False)

In [ ]:
pos = 1220
control = 4898

## All Single Model Dev

In [ ]:
args.str_all_cols = pickle.load(open(args.path/'str_all_cols.pkl', 'rb'))
args.all_cols=['hadm_id'] + args.str_all_cols + ['note', 'imi_adm_label']

In [ ]:
full_common_all = pd.read_csv(args.all_csv, usecols=args.all_cols)
full_common_all = full_common_all[full_common_all['imi_adm_label'] != -1].reset_index(drop=True)
notes_common_all = full_common_all[['hadm_id', 'note', 'imi_adm_label']].copy().reset_index(drop=True)

print(notes_common_all.shape, full_common_all.shape)
notes_common_all['hadm_id'].nunique(), full_common_all['hadm_id'].nunique()

In [ ]:
# notes_common_all.dropna(inplace=True)
# notes_common_all.reset_index(drop=True, inplace=True)

# all_hadms = set(full_common_all['hadm_id'].unique()).intersection(notes_common_all['hadm_id'].unique())

# full_common_all = full_common_all[full_common_all['hadm_id'].isin(all_hadms)].reset_index(drop=True)
# notes_common_all = notes_common_all[notes_common_all['hadm_id'].isin(all_hadms)].reset_index(drop=True)

# print(notes_common_all.shape, full_common_all.shape)
# notes_common_all['hadm_id'].nunique(), full_common_all['hadm_id'].nunique()

In [ ]:
# full_common_all.dropna(inplace=True)
# full_common_all.reset_index(drop=True, inplace=True)

In [ ]:
# full_common_all[['note']] = full_common_all.groupby('hadm_id')[['note']].ffill()

# full_common_all.dropna(inplace=True)
# full_common_all.reset_index(drop=True, inplace=True)

# x = pd.DataFrame(full_common_all.isna().sum(), columns=['sum']).reset_index()
# x['sum'].sum()

### Notes only

In [ ]:
df = set_group_splits(notes_common_all.copy(), group_col='hadm_id', seed=seed)

train_df = df[df['split'] == 'train']
test_df = df[df['split'] == 'test']

g = train_df.groupby(['imi_adm_label']).size().to_numpy()
print(f"Prevalence of positive class in training set:{(g[1]/g.sum())*100:0.1f}%")
g = test_df.groupby(['imi_adm_label']).size().to_numpy()
print(f"Prevalence of positive class in test set:{(g[1]/g.sum())*100:0.1f}%")

In [ ]:
# vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=60_000)
# x_note_train = vectorizer.fit_transform(train_df['note'])
# x_note_test = vectorizer.transform(test_df['note'])

# with open(args.workdir/f'vectordir/bi_gram_all_643.pkl', 'wb') as f:
#   pickle.dump(vectorizer, f)
#   pickle.dump(x_note_train, f)
#   pickle.dump(x_note_test, f)

In [ ]:
with open(args.workdir/f'vectordir/bi_gram_all_643.pkl', 'rb') as f:
  vectorizer = pickle.load(f)
  x_note_train = pickle.load(f)
  x_note_test = pickle.load(f)
  
y_train, y_test = train_df['imi_adm_label'], test_df['imi_adm_label']
x_note_train.shape, x_note_test.shape, y_train.shape, y_test.shape

In [ ]:
clf = LogisticRegression(class_weight='balanced')
clf.fit(x_note_train, y_train)

prob = clf.predict_proba(x_note_test)
pos_prob = prob[:, 1]

labels = ['Delayed', 'Imminent']
label_test = [labels[i] for i in y_test]

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
plot_roc(label_test, prob, title='', ax=ax)
ax.set_xlabel('1 - Specificity')
ax.set_ylabel('Sensitivity')

In [ ]:
save = True

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
plot_thresh_range(ax, y_test, pos_prob, lower=0.1, upper=0.81, n_vals=100)

if save:
  fig.savefig(args.figdir/f'{args.model}_notes_all_metrics_vary.pdf', dpi=300)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
best_threshold = plot_thresh_metric(ax, y_test, pos_prob, lower=0.1, upper=0.81, n_vals=100)
ax.text(0.71, ax.get_ylim()[1] * 0.9, f'Optimum Threshold = {best_threshold[0]}', fontsize=12, color='b')
print(best_threshold)

if save:
  fig.savefig(args.figdir/f'{args.model}_notes_all_threshold_guide.pdf', dpi=300)

In [ ]:
threshold = 0.46
pred = (pos_prob > threshold).astype(np.int64)
label_preds = [labels[i] for i in pred]
cm = confusion_matrix(y_test, pred)
tn,fp,fn,tp = cm[0][0],cm[0][1],cm[1][0],cm[1][1]
sensitivity = tp/(tp+fn)
specificity = tn/(tn+fp)
ppv = tp/(tp+fp)
npv = tn/(tn+fn)
f1 = (2*ppv*sensitivity)/(ppv+sensitivity)
auroc = roc_auc_score(y_test, pos_prob)

d = {
  'sensitivity': np.round(sensitivity, 3),
  'specificity': np.round(specificity, 3),
  'ppv': np.round(ppv, 3),
  'npv': np.round(npv, 3),
  'f1': np.round(f1, 3),
  'auroc': np.round(auroc, 3),
  'threshold': threshold,
}
metrics = pd.DataFrame(d.values(), index=d.keys(), columns=['Value'])
metrics

In [ ]:
fig, ax = plt.subplots(figsize=(11, 8))
plot_confusion_matrix(label_test, label_preds, x_tick_rotation=45, ax=ax, normalize=True)

### Notes and All structured

In [ ]:
df = set_group_splits(full_common_all.copy(), group_col='hadm_id', seed=seed)

train_df = df[df['split'] == 'train']
test_df = df[df['split'] == 'test']

g = train_df.groupby(['imi_adm_label']).size().to_numpy()
print(f"Prevalence of positive class in training set:{(g[1]/g.sum())*100:0.1f}%")
g = test_df.groupby(['imi_adm_label']).size().to_numpy()
print(f"Prevalence of positive class in test set:{(g[1]/g.sum())*100:0.1f}%")

In [ ]:
# vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=60_000)
# x_note_train = vectorizer.fit_transform(train_df['note'])
# x_note_test = vectorizer.transform(test_df['note'])

# with open(args.workdir/f'vectordir/bi_gram_full_vital_643.pkl', 'wb') as f:
#   pickle.dump(vectorizer, f)
#   pickle.dump(x_note_train, f)
#   pickle.dump(x_note_test, f)

In [ ]:
with open(args.workdir/f'vectordir/bi_gram_all_643.pkl', 'rb') as f:
  vectorizer = pickle.load(f)
  x_note_train = pickle.load(f)
  x_note_test = pickle.load(f)
  
y_train, y_test = train_df['imi_adm_label'], test_df['imi_adm_label']
x_note_train.shape, x_note_test.shape, y_train.shape, y_test.shape

In [ ]:
x_str_train, x_str_test = train_df[args.str_all_cols].values, test_df[args.str_all_cols].values
x_str_train.shape, x_str_test.shape

In [ ]:
x_train = scipy.sparse.hstack((x_str_train, x_note_train)).tocsr()
x_test = scipy.sparse.hstack((x_str_test, x_note_test)).tocsr()
x_train.shape, x_test.shape

In [ ]:
clf = LogisticRegression(class_weight='balanced')
clf.fit(x_train, y_train)

prob = clf.predict_proba(x_test)
pos_prob = prob[:, 1]

labels = ['Delayed', 'Imminent']
label_test = [labels[i] for i in y_test]

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
plot_roc(label_test, prob, title='', ax=ax)
ax.set_xlabel('1 - Specificity')
ax.set_ylabel('Sensitivity')

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
plot_thresh_range(ax, y_test, pos_prob, lower=0.1, upper=0.81, n_vals=100)

if save:
  fig.savefig(args.figdir/f'{args.model}_full_all_metrics_vary.pdf', dpi=300)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
best_threshold = plot_thresh_metric(ax, y_test, pos_prob, lower=0.1, upper=0.81, n_vals=100)
ax.text(0.71, ax.get_ylim()[1] * 0.9, f'Optimum Threshold = {best_threshold[0]}', fontsize=12, color='b')
print(best_threshold)

if save:
  fig.savefig(args.figdir/f'{args.model}_full_all_threshold_guide.pdf', dpi=300)

In [ ]:
threshold = 0.5
pred = (pos_prob > threshold).astype(np.int64)
label_preds = [labels[i] for i in pred]
cm = confusion_matrix(y_test, pred)
tn,fp,fn,tp = cm[0][0],cm[0][1],cm[1][0],cm[1][1]
sensitivity = tp/(tp+fn)
specificity = tn/(tn+fp)
ppv = tp/(tp+fp)
npv = tn/(tn+fn)
f1 = (2*ppv*sensitivity)/(ppv+sensitivity)
auroc = roc_auc_score(y_test, pos_prob)

d = {
  'sensitivity': np.round(sensitivity, 3),
  'specificity': np.round(specificity, 3),
  'ppv': np.round(ppv, 3),
  'npv': np.round(npv, 3),
  'f1': np.round(f1, 3),
  'auroc': np.round(auroc, 3),
  'threshold': threshold,
}
metrics = pd.DataFrame(d.values(), index=d.keys(), columns=['Value'])
metrics

In [ ]:
fig, ax = plt.subplots(figsize=(11, 8))
plot_confusion_matrix(label_test, label_preds, x_tick_rotation=45, ax=ax, normalize=False)

In [ ]:
pos = 1220
control = 4898

## Metrics

In [ ]:
with open(args.workdir/f'preds.pkl', 'rb') as f:
  targs = pickle.load(f)
  probs = pickle.load(f)
  preds = pickle.load(f)

bam = BinaryAvgMetrics(targs, preds, [prob[:, 1] for prob in probs])
bam.get_avg_metrics(defn=True)

In [ ]:
bam.get_avg_metrics(conf=0.95)

In [ ]:
fig, ax = plt.subplots(figsize=(11, 8))
plot_cm(ax, bam.cm_avg, ['Delayed', 'Imminent'])

if save:
  fig.savefig(args.figdir/f'mean_cm.pdf', dpi=300)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
plot_mean_roc(ax, bam.targs, probs)

if save:
  fig.savefig(args.figdir/f'mean_roc.pdf', dpi=300)